In [ ]:
from google.colab import drive
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn import model_selection
from nltk.tokenize import word_tokenize
import nltk
from sklearn.metrics import accuracy_score
nltk.download('punkt')
import codecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.82.235.242:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, -5844535341722424470),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -4550017439063403169),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 413606623537819845),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8341141000206418897),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -6709762387259065559),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -2644875381155987240),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 8156336762412113320),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, -7312521236575599346),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.82.235.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.82.235.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2021-06-16 07:49:40--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  28.1MB/s    in 23s     

2021-06-16 07:50:04 (27.7 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
!unzip wiki-news-300d-1M.vec.zip

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
class FasttextModels:
  def __init__(self,path):
    df = pd.read_csv(path)
    df['text'] = df['text'].apply(lambda x: str(x))
    self.dfs_train = df[df['kfold']!=0]
    self.dfs_valid = df[df['kfold']==0]



  def process(self):
    self.read_fasttext()
    self.data_per_fold_fasttext()
    return self.fasttext_wt,self.lls
    

  def read_fasttext(self):
      embeddings_index = {}
      f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')
      for line in f:
          values = line.rstrip().rsplit(' ')
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()
      print('found %s word vectors' % len(embeddings_index))
      self.fasttext_embeddings_index = embeddings_index
  
  def data_per_fold_fasttext(self):
      # self.data_fasttext = {}
      embed_dim = 300
      self.data_fasttext = self.create_fasttext_embedding_matrix(embed_dim)

  def create_fasttext_embedding_matrix(self,embed_dim):
      train = self.dfs_train
      test = self.dfs_valid
      x_train = train.text.values
      y_train = train.lbl_num.values
      x_test = test.text.values
      y_test = test.lbl_num.values
      word_tokenizer = Tokenizer(num_words = 50000,oov_token="<OOV>")
      word_tokenizer.fit_on_texts(x_train)
      vocab_length = len(word_tokenizer.word_index) + 1
      x_train_emb = word_tokenizer.texts_to_sequences(x_train)
      x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
      word_count = lambda sentence: len(word_tokenize(sentence))
      longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
      length_long_sentence = len(word_tokenize(longest_sentence))

      self.fasttext_wt = word_tokenizer
      self.lls = length_long_sentence
      x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
      x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

      # encoded_Y_tr = self.encoder.fit_transform(y_train)
      # encoded_Y_ts = self.encoder.fit_transform(y_test)
      dummy_y_tr = np_utils.to_categorical(y_train)
      dummy_y_ts = np_utils.to_categorical(y_test)
      # self.df_labels = pd.DataFrame.from_dict({'y_train':y_train,'y_test':y_test,'dummy_tr':dummy_y_tr,'dummy_y_ts':dummy_y_ts})

      embedding_matrix = np.zeros((vocab_length, embed_dim))
      for word, index in word_tokenizer.word_index.items():
          embedding_vector = self.fasttext_embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector

      # print(x_train_padded)
      # print(x_test_padded)
      return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))

  def CONV1D(self,v):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300D_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n CONV1D \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/FT_300/sentiment/sentiment_CONV1D.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his#,accuracy_score(y_ts,y_pred)



  def CONV1D_BiLSTM(self,v):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300D_CONV1D_BiLSTM_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_CONV1D_BiLSTM_embedding"),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n CONV1D BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/FT_300/sentiment/sentiment_CONV1D_BiLSTM.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)


    
  
  def BiGRU(self,v):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300D_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiGRU_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiGRU_embedding"),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/FT_300/sentiment/sentiment_BiGRU.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his #,accuracy_score(y_ts,y_pred)
      
  def BiLSTM(self,v):
    data = self.data_fasttext
    x_tr,y_tr = data[0]
    x_ts,y_ts = data[1]
    vl,mat,dim,len_sen = data[2]
    
    with strategy.scope():
        model = tf.keras.models.Sequential([
            tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300D_BiLSTM_embedding"),
            # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_BiLSTM_embedding"),
            # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_BiLSTM_embedding"),
            # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
            # tf.keras.layers.MaxPool1D(),
            # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
            # tf.keras.layers.MaxPool1D(),
            # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
            # tf.keras.layers.MaxPool1D(),
            # tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='sigmoid'),
            tf.keras.layers.Dense(256, activation='sigmoid'),
            tf.keras.layers.Dense(128, activation='sigmoid'),
            tf.keras.layers.Dense(64, activation='sigmoid'),
            # tf.keras.layers.Dense(5, activation='softmax')
            tf.keras.layers.Dense(v, activation='softmax')])

        model.compile(loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy'])

    print("\n BiLSTM \n")
    model.summary()
    his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/FT_300/sentiment/sentiment_BiLSTM.h5', monitor='val_loss', save_best_only=True)])
    # y_pred = model.predict(x_ts)
    return model,his#,accuracy_score(y_ts,y_pred)

  def SEP_CONV1D(self,v):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300D_SEP_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="GL_200_SEP_CONV1D_embedding"),
              # tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False,name="FT_300_SEP_CONV1D_embedding"),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(v, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      # x = model.to_json()
      print("\n SEPERABLE CONV1D \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping(monitor='val_loss',patience=2),ModelCheckpoint(filepath='/content/drive/MyDrive/FT_300/sentiment/sentiment_SEP_CONV1D.h5', monitor='val_loss', save_best_only=True)])
      # y_pred = model.predict(x_ts)
      return model,his


In [ ]:
sent_models = {}
sent_models_clf = FasttextModels('/content/drive/MyDrive/datasets/tr_sen_fin.csv')
sent_param_fasttext = {}
sent_param_fasttext['Tokenizer'],sent_param_fasttext['long_sen_num'] = sent_models_clf.process()
# sent_models['CONV1D'] = sent_models_clf.CONV1D(5)
# sent_models['CONV1D_BiLSTM'] = sent_models_clf.CONV1D_BiLSTM(5)
# # # sent_models['CONV1D_BiGRU'] = sent_models_clf.CONV1D_BiGRU(2)
# sent_models['BiLSTM'] = sent_models_clf.BiLSTM(5)
# sent_models['BiGRU'] = sent_models_clf.BiGRU(5)
sent_models['SEP_CONV1D'] = sent_models_clf.SEP_CONV1D(5)


found 999995 word vectors

 SEPERABLE CONV1D 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FT_300D_SEP_CONV1D_embedding (None, 163, 300)          41135100  
_________________________________________________________________
separable_conv1d (SeparableC (None, 161, 32)           10532     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 80, 32)            0         
_________________________________________________________________
separable_conv1d_1 (Separabl (None, 78, 32)            1152      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 39, 32)            0         
_________________________________________________________________
separable_conv1d_2 (Separabl (None, 37, 32)            1152      
_________________________________________________________________
max_poo

In [ ]:
import pickle
with open('/content/drive/MyDrive/FT_300/sentiment/sentiment_params.pickle','wb') as f:
  pickle.dump(sent_param_fasttext,f)